In [1]:
import csv
import numpy as np
from scipy.sparse.linalg import eigs
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# CONFIG
# Don't change this unless testing different dataset.
adj_filename = 'data/PEMS04.csv'
traffic_filename = 'data/PEMS04.npz'
weather_filename = 'data/weather.npz'

# Note that num_X is a MULTIPLE of the prediction window
# num_hours only does what it says if pred_window is an hour
# otherwise, num_hours = 3 and window_size = 24 means it's 3 2-hour periods
num_hours = 2
num_days = 2
num_weeks = 2

# measured in 5 minute intervals
# the code is not built to handle unreasonable window sizes
# please limit your window size to the scope of several hours at worst
pred_window_size = 12

# Depth for convolution - represents neighbor depth.
# Keep low to avoid slow model. 3 should be a reasonable limit.
K = 3

epochs = 1
learning_rate = 0.01

blocks = 1

gcn_filters = 2
t_filters = 2


In [3]:
# Global variables
# timestamps_per_hour is equivalent to prediction window size
timestamps_per_week = 2016
timestamps_per_day = 288

# returns filepath for specific parameters
def get_filepath(num_hours, num_days, num_weeks):
    return f'data/packaged/{num_hours}_{num_days}_{num_weeks}.npz'
    
def get_required_diff(hours, days, weeks, pred_window_size):
    '''
    Returns the window size of one data point
    This window will metaphorically slide across the dataset, generating data
    '''
    
    # Typically you would expect the week to be the longest.
    # This is designed to accomodate for faulty input.
    
    week_window_size = timestamps_per_week * weeks
    day_window_size = timestamps_per_day * days
    hour_window_size = pred_window_size * hours
    return max(week_window_size, day_window_size, hour_window_size) + pred_window_size

def generate_traffic_data(file, hours, days, weeks, pred_window_size):
    '''
    Returns four arrays of dims ([S, N, F, T_h], [S, N, F, T_d], [S, N, F, T_w], [S, N, T_p])
        S   -> number of datapoints total
        N   -> number of nodes. For PEMS04, this is 307
        F   -> number of features per node. For PEMS04, this is 3
        T_h -> timestamps for hourly channel
        T_d -> timestamps for daily channel
        T_w -> timestamps for weekly channel
        T_p -> timestamps for prediction output
    '''
    
    data = np.load(file)['data'] # [16992, 307, 3]
    
    # Get range of possible start/stops given the data
    window_size = get_required_diff(hours, days, weeks, pred_window_size)
    num_datapoints = data.shape[0] - window_size + 1
    
    print(f'Window size: {window_size}')
    print(f'Generating {num_datapoints} datapoints')
    
    # output arrays
    X_h = []
    X_d = []
    X_w = []
    y = []
    
    for i in range(num_datapoints):
        t_0 = i + window_size - pred_window_size
        
        temp_xh = []
        temp_xd = []
        temp_xw = []
        temp_y = []
        
        # Hourly
        for hour in range(hours, 0, -1):
            start = t_0 - hour * pred_window_size
            for j in range(pred_window_size):
                temp_xh.append(data[start + j])
        
        # Daily
        for day in range(days, 0, -1):
            start = t_0 - day * timestamps_per_day
            for j in range(pred_window_size):
                temp_xd.append(data[start + j])
            
        # Weekly
        for week in range(weeks, 0, -1):
            start = t_0 - week * timestamps_per_week
            for j in range(pred_window_size):
                temp_xw.append(data[start + j])
                
        # Expected output
        for j in range(pred_window_size):
            temp_y.append(data[t_0 + j][:,1])
            
        # Reshape to fit data
        if hours > 0:
            temp_xh = np.array(temp_xh, dtype = np.float32).transpose(1, 2, 0)
        if days > 0:
            temp_xd = np.array(temp_xd, dtype = np.float32).transpose(1, 2, 0)
        if weeks > 0:
            temp_xw = np.array(temp_xw, dtype = np.float32).transpose(1, 2, 0)
        temp_y = np.array(temp_y, dtype = np.float32).transpose(1, 0)
        
        X_h.append(temp_xh)
        X_d.append(temp_xd)
        X_w.append(temp_xw)
        y.append(temp_y)
        
    X_h = np.array(X_h, dtype = np.float32)
    X_d = np.array(X_d, dtype = np.float32)
    X_w = np.array(X_w, dtype = np.float32)
    y = np.array(y, dtype = np.float32)
        
    print(f'X_h size: {X_h.shape}')
    print(f'X_d size: {X_d.shape}')
    print(f'X_w size: {X_w.shape}')
    print(f'y size: {y.shape}')
    print('')
    
    return (X_h, X_d, X_w, y)
    
def generate_weather_data(file, hours, days, weeks, pred_window_size):
    data = np.load(file)['data']
    
    # Get range of possible start/stops given the data
    window_size = get_required_diff(hours, days, weeks, pred_window_size)
    num_datapoints = data.shape[0] - window_size + 1
    
    print(f'Window size: {window_size}')
    print(f'Generating {num_datapoints} datapoints')
    
    W = []
    
    for i in range(num_datapoints):
        t_0 = i + window_size - pred_window_size

        temp_w = []
        for j in range(pred_window_size):
            temp_w.append(data[t_0 + j])
            
        temp_w = np.array(temp_w, dtype = np.float32).transpose(1, 0)
        W.append(temp_w)
        
    W = np.array(W, dtype = np.float32)
    
    print(f'W size: {W.shape}')
    
    return W

def generate_adjacency_matrix(adj_file, data_file):
        data = np.load(data_file)['data']
        num_nodes = data.shape[1]
        
        print('Generating adjacency matrix for traffic nodes.')
        print(f'Number of nodes: {num_nodes}')
        
        # Initialize adjacency matrix
        A = np.zeros((num_nodes, num_nodes), dtype=np.float32)
        print(f'Shape of adjacency matrix: {A.shape}')
        
        # Fill in adjacency matrix
        with open(adj_file, 'r') as f:
            f.readline()
            reader = csv.reader(f)
            for row in reader:
                if len(row) != 3:
                    continue
                i, j = int(row[0]), int(row[1])
                A[i, j] = 1
                
        return A


In [4]:
# print(f'Adjacency matrix filename: {adj_filename}.')
# print(f'Traffic data filename: {traffic_filename}.')
# print(f'Weather data filename: {weather_filename}')
# print('')

# #================
# # Retrieving data
# #================

# print(f'Predicting using information from {num_hours} hours, {num_days} days, and {num_weeks} weeks prior to prediction period.')

# savefile = get_filepath(num_hours, num_days, num_weeks)

# print(f'Data will be saved to {savefile}')
# print('')

# # Retrieve generic pool of traffic data
# X_h, X_d, X_w, y = generate_traffic_data(traffic_filename, num_hours, num_days, num_weeks, pred_window_size)

# print('Successfully generated traffic dataset')

# # Retrieve weather data
# W = generate_weather_data(weather_filename, num_hours, num_days, num_weeks, pred_window_size)

# print('Successfully generated weather dataset')

# # Retrieve adjacency matrix
# A = generate_adjacency_matrix(adj_filename, traffic_filename)

# print('Successfully generated adjacency matrix')

# #============
# # Saving data
# #============

# np.savez_compressed(savefile, hourly = X_h, daily = X_d, weekly = X_w, pred = y, weather = W, adj_mx = A)
# print('Successfully saved.')

In [5]:
def scaled_Laplacian(W):
    D = np.diag(np.sum(W, axis = 1))
    L = D - W
    lambda_max = eigs(L, k = 1, which = 'LR')[0].real
    return (2 * L) / lambda_max - np.identity(W.shape[0])
    
def cheb_polynomial(L_tilde, K):
    N = L_tilde.shape[0]
    cheb_polynomials = [np.identity(N), L_tilde.copy()]
    for i in range(2, K):
        cheb_polynomials.append(2 * L_tilde * cheb_polynomials[i - 1] - cheb_polynomials[i - 2])
    return cheb_polynomials

class TAT(nn.Module):
    def __init__(self, inputs, vertices, timesteps):
        super(TAT, self).__init__()
        self.U1 = nn.Parameter(torch.FloatTensor(vertices))
        self.U2 = nn.Parameter(torch.FloatTensor(inputs, vertices))
        self.U3 = nn.Parameter(torch.FloatTensor(inputs))
        self.be = nn.Parameter(torch.FloatTensor(1, timesteps, timesteps))
        self.Ve = nn.Parameter(torch.FloatTensor(timesteps, timesteps))

    def forward(self, x):
        inner = torch.matmul(x.permute(0, 3, 2, 1), self.U1)
        lhs = torch.matmul(inner, self.U2)
        rhs = torch.matmul(self.U3, x)
        product = torch.matmul(lhs, rhs)
        E = torch.matmul(self.Ve, torch.sigmoid(product + self.be))
        E_normalized = F.softmax(E, dim=1)
        return E_normalized

class SAT(nn.Module):
    def __init__(self, inputs, vertices, timesteps):
        super(SAT, self).__init__()
        self.W1 = nn.Parameter(torch.FloatTensor(timesteps))
        self.W2 = nn.Parameter(torch.FloatTensor(inputs, timesteps))
        self.W3 = nn.Parameter(torch.FloatTensor(inputs))
        self.bs = nn.Parameter(torch.FloatTensor(1, vertices, vertices))
        self.Vs = nn.Parameter(torch.FloatTensor(vertices, vertices))
        
    def forward(self, x):
        lhs = torch.matmul(torch.matmul(x, self.W1), self.W2)
        rhs = torch.matmul(self.W3, x).transpose(-1, -2)
        product = torch.matmul(lhs, rhs)
        S = torch.matmul(self.Vs, torch.sigmoid(product + self.bs))
        S_normalized = F.softmax(S, dim=1)

        return S_normalized

class SAT_Conv(nn.Module):
    def __init__(self, K, cheb_polynomials, inputs, outputs):
        super(SAT_Conv, self).__init__()
        self.K = K
        self.cheb_polynomials = cheb_polynomials
        self.inputs = inputs
        self.outputs = outputs
        self.Theta = nn.ParameterList([nn.Parameter(torch.FloatTensor(inputs, outputs)) for _ in range(K)])
        
    def forward(self, x, sat):
        batch_size, vertices, inputs, timesteps = x.shape

        final_outputs = []

        for t in range(timesteps):

            graph_signal = x[:, :, :, t]
            output = torch.zeros(batch_size, vertices, self.outputs)

            for k in range(self.K):
                T_k = self.cheb_polynomials[k]
                T_k_with_at = T_k.mul(sat)
                theta_k = self.Theta[k]
                rhs = T_k_with_at.permute(0, 2, 1).matmul(graph_signal)
                output = output + rhs.matmul(theta_k)

            final_outputs.append(output.unsqueeze(-1))

        return F.relu(torch.cat(final_outputs, dim=-1))

class ASTGCN_Block(nn.Module):
    def __init__(self, inputs, K, chev_filters, time_filters, strides, cheb_polynomials, vertices, timesteps):
        super(ASTGCN_Block, self).__init__()
        self.TAt = TAT(inputs, vertices, timesteps)
        self.SAt = SAT(inputs, vertices, timesteps)
        self.SAt_conv = SAT_Conv(K, cheb_polynomials, inputs, chev_filters)
        self.t_conv = nn.Conv2d(chev_filters, time_filters, kernel_size = (1, 3), stride = (1, strides), padding = (0, 1))
        self.residual_conv = nn.Conv2d(inputs, time_filters, kernel_size = (1, 1), stride = (1, strides))
        self.ln = nn.LayerNorm(time_filters)

    def forward(self, x):
        batch_size, vertices, features, timesteps = x.shape
        tat = self.TAt(x)
        x_TAt = torch.matmul(x.reshape(batch_size, -1, timesteps), tat).reshape(batch_size, vertices, features, timesteps)
        sat = self.SAt(x_TAt)
        gcn = self.SAt_conv(x, sat)
        t_conv_output = self.t_conv(gcn.permute(0, 2, 1, 3))
        x_residual = self.residual_conv(x.permute(0, 2, 1, 3))
        x_residual = self.ln(F.relu(x_residual + t_conv_output).permute(0, 3, 2, 1)).permute(0, 2, 3, 1)
        return x_residual

class ASTGCN_Module(nn.Module):
    def __init__(self, blocks, traffic_features, K, chev_filters, time_filters, strides, cheb_polynomials, outputs, vertices):
        super(ASTGCN_Module, self).__init__()

        self.BlockList = nn.ModuleList([ASTGCN_Block(traffic_features, K, chev_filters, time_filters, strides, cheb_polynomials, vertices, strides * outputs)])

        self.BlockList.extend([ASTGCN_Block(time_filters, K, chev_filters, time_filters, 1, cheb_polynomials, vertices, outputs) for _ in range(blocks - 1)])

        self.final_conv = nn.Conv2d(outputs, outputs, kernel_size = (1, time_filters))
        
    def forward(self, x):
        for block in self.BlockList:
            x = block(x)
        output = self.final_conv(x.permute(0, 3, 1, 2))[:, :, :, -1].permute(0, 2, 1)
        return output

class Weather(nn.Module):
    def __init__(self, weather_features, outputs):
        super(Weather, self).__init__()
        self.layer = nn.Linear(weather_features, outputs)
        
    def forward(self, W):
        W = W.permute(0, 2, 1)
        W = self.layer(W)
        W = W.permute(0, 2, 1)
        return W
    
class ASTGCN(nn.Module):
    def __init__(self, blocks, traffic_features, K, chev_filters, time_filters, cheb_polynomials, outputs, hours, days, weeks, vertices, weather_features):
        super(ASTGCN, self).__init__()
        self.hourly = ASTGCN_Module(blocks, traffic_features, K, chev_filters, time_filters, hours, cheb_polynomials, outputs, vertices)
        self.daily = ASTGCN_Module(blocks, traffic_features, K, chev_filters, time_filters, days, cheb_polynomials, outputs, vertices)
        self.weekly = ASTGCN_Module(blocks, traffic_features, K, chev_filters, time_filters, weeks, cheb_polynomials, outputs, vertices)
        self.weather = Weather(weather_features, vertices)
        self.W_h = nn.Parameter(torch.FloatTensor(vertices, outputs))
        self.W_d = nn.Parameter(torch.FloatTensor(vertices, outputs))
        self.W_w = nn.Parameter(torch.FloatTensor(vertices, outputs))
        self.W_weather = nn.Parameter(torch.FloatTensor(vertices, outputs))
        self.b = nn.Parameter(torch.FloatTensor(vertices, outputs))
        
    def forward(self, X_h, X_d, X_w, W):
        h_out = self.hourly(X_h)
        d_out = self.daily(X_d)
        w_out = self.weekly(X_w)
        weather_out = self.weather(W)
        fusion = self.W_h.mul(h_out) + self.W_d.mul(d_out) + self.W_w.mul(w_out) + self.W_weather.mul(weather_out) + self.b
        
        return fusion

def get_model(blocks, traffic_features, K, gcn_filters, t_filters, pred_window, hours, days, weeks, vertices, weather_features, A):
    L_tilde = scaled_Laplacian(A)
    cheb_polynomials = [torch.from_numpy(i).type(torch.FloatTensor) for i in cheb_polynomial(L_tilde, K)]
    return ASTGCN(blocks, traffic_features, K, gcn_filters, t_filters, cheb_polynomials, pred_window, hours, days, weeks, vertices, weather_features)


In [6]:
def train_one_epoch(model, training_loader, loss_function, optimizer):
    mae = 0
    rmse = 0

    for i, data in enumerate(training_loader):
        print(f'Training batch {i + 1}', end  = '\r')
        train_xh, train_xd, train_xw, train_w, train_y = data
        optimizer.zero_grad()
        pred_y = model(train_xh, train_xd, train_xw, train_w)
        loss = torch.sqrt(loss_function(pred_y, train_y))
        loss.backward()
        optimizer.step()
                    
        mae += mean_absolute_error(train_y.detach().numpy().reshape(-1, 1), pred_y.detach().numpy().reshape(-1, 1))
        rmse += mean_squared_error(train_y.detach().numpy().reshape(-1, 1), pred_y.detach().numpy().reshape(-1, 1), squared = True)
        
    mae = mae / (i + 1)
    rmse = rmse / (i + 1)
    print(f'TRAINING: MAE = {mae} RMSE = {rmse}')
    return

def train_test(model, training_loader, testing_loader, num_epochs, lr):
    loss_function = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr =  0.01)
    
    for epoch in range(num_epochs):
        print(f'BEGIN: Epoch {epoch + 1}')
        
        for p in model.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
            else:
                nn.init.uniform_(p)
        
        print('Done initializing parameters.')
        
        model.train(True)
        
        avg_loss = train_one_epoch(model, training_loader, loss_function, optimizer)
        
        model.train(False)
        
        t_loss = 0
        
        # Testing
        mae = 0
        rmse = 0
        for i, data in enumerate(testing_loader):
            print(f'Testing batch {i + 1}', end  = '\r')
            test_xh, test_xd, test_xw, test_w, test_y = data
            pred_y = model(test_xh, test_xd, test_xw, test_w)
            mae += mean_absolute_error(test_y.detach().numpy().reshape(-1, 1), pred_y.detach().numpy().reshape(-1, 1))
            rmse += mean_squared_error(test_y.detach().numpy().reshape(-1, 1), pred_y.detach().numpy().reshape(-1, 1), squared = True)
            print(test_xh.shape)
            
        mae = mae / (i + 1)
        rmse = rmse / (i + 1)
        print(f'EPOCH {epoch + 1}: MAE = {mae} RMSE = {rmse}')

In [ ]:
savefile = get_filepath(num_hours, num_days, num_weeks)
    
print(f'Retreiving data from {savefile}')

packaged_data = np.load(savefile)

# X_h = torch.from_numpy(packaged_data['hourly'])
# X_d = torch.from_numpy(packaged_data['daily'])
# X_w = torch.from_numpy(packaged_data['weekly'])
# W = torch.from_numpy(packaged_data['weather'])
# y = torch.from_numpy(packaged_data['pred'])

# A = torch.from_numpy(packaged_data['adj_mx'])

X_h = np.copy(packaged_data['hourly'])
X_d = np.copy(packaged_data['daily'])
X_w = np.copy(packaged_data['weekly'])
W = np.copy(packaged_data['weather'])
y = np.copy(packaged_data['pred'])

A = np.copy(packaged_data['adj_mx'])

traffic_features = X_h.shape[2]
weather_features = W.shape[1]
vertices = y.shape[1]

packaged_data = None

print(f'Hourly traffic dims: {X_h.shape}')
print(f'Daily traffic dims: {X_d.shape}')
print(f'Weekly traffic dims: {X_w.shape}')
print(f'Weather dims: {W.shape}')
print(f'Prediction dims: {y.shape}')
print(f'Adjacency matrix dims: {A.shape}')
print(f'Number of traffic features: {traffic_features}')
print(f'Number of weather features: {weather_features}')
print(f'Number of vertices: {vertices}')
print('')

#===============
# Data splitting
#===============

data_indices = list(range(y.shape[0]))

train_ind, test_ind = train_test_split(data_indices)

print(f'Training size: {len(train_ind)}')
print(f'Testing size: {len(test_ind)}')

print('X_h')
train_xh = torch.from_numpy(np.copy(X_h[train_ind]))
test_xh = torch.from_numpy(np.copy(X_h[test_ind]))
X_h = None
print('X_d')
train_xd = torch.from_numpy(np.copy(X_d[train_ind]))
test_xd = torch.from_numpy(np.copy(X_d[test_ind]))
X_d = None
print('X_w')
train_xw = torch.from_numpy(np.copy(X_w[train_ind]))
test_xw = torch.from_numpy(np.copy(X_w[test_ind]))
X_w = None
print('W')
train_w = torch.from_numpy(np.copy(W[train_ind]))
test_w = torch.from_numpy(np.copy(W[test_ind]))
W = None
print('y')
train_y = torch.from_numpy(np.copy(y[train_ind]))
test_y = torch.from_numpy(np.copy(y[test_ind]))
y = None


print(f'Training X_h shape: {train_xh.shape}')
print(f'Testing X_h shape: {test_xh.shape}')
print(f'Training X_d shape: {train_xd.shape}')
print(f'Testing X_d shape: {test_xd.shape}')
print(f'Training X_w shape: {train_xw.shape}')
print(f'Testing X_w shape: {test_xw.shape}')
print(f'Training W shape: {train_w.shape}')
print(f'Testing W shape: {test_w.shape}')
print(f'Training y shape: {train_y.shape}')
print(f'Testing y shape: {test_y.shape}')
print('')

training_dataset = TensorDataset(train_xh, train_xd, train_xw, train_w, train_y)
testing_dataset = TensorDataset(test_xh, test_xd, test_xw, test_w, test_y)

training_loader = DataLoader(training_dataset, batch_size = 12, shuffle = True)
testing_loader = DataLoader(testing_dataset, batch_size = 12)

train_xh = None
test_xh = None
train_xd = None
test_xd = None
train_xw = None
test_xw = None
train_w = None
test_w = None
train_y = None
test_y = None

print(f'Training loader size: {len(training_loader)}')
print(f'Testing loader size: {len(testing_loader)}')

#===============
# Model Creation
#===============

# TODO make model
model = get_model(blocks, traffic_features, K, gcn_filters, t_filters, pred_window_size, num_hours, num_days, num_weeks, vertices, weather_features, A)
# model = Weather(weather_features, pred_window_size)

#===================
# Training + Testing
#===================
train_test(model, training_loader, testing_loader, epochs, learning_rate)



Retreiving data from data/packaged/2_2_2.npz
Hourly traffic dims: (12949, 307, 3, 24)
Daily traffic dims: (12949, 307, 3, 24)
Weekly traffic dims: (12949, 307, 3, 24)
Weather dims: (12949, 16, 12)
Prediction dims: (12949, 307, 12)
Adjacency matrix dims: (307, 307)
Number of traffic features: 3
Number of weather features: 16
Number of vertices: 307

Training size: 9711
Testing size: 3238
X_h
X_d
X_w
W
y
